In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow

In [38]:
csv_url = (
        'https://github.com/samfennelly/mlflowdemo/blob/main/titanic_train.csv?raw=true'
)
data = pd.read_csv(csv_url)
data = data.fillna(0)
data

,passenger_id,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,survived
0,1216,3,"Smyth, Miss. Julia",female,0.0,0,0,335432,7.7333,0,Q,13,0.0,0,1
1,699,3,"Cacic, Mr. Luka",male,38.0,0,0,315089,8.6625,0,S,0,0.0,Croatia,0
2,1267,3,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",female,30.0,1,1,345773,24.1500,0,S,0,0.0,0,0
3,449,2,"Hocking, Mrs. Elizabeth (Eliza Needs)",female,54.0,1,3,29105,23.0000,0,S,4,0.0,"Cornwall / Akron, OH",1
4,576,2,"Veal, Mr. James",male,40.0,0,0,28221,13.0000,0,S,0,0.0,"Barre, Co Washington, VT",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,158,1,"Hipkins, Mr. William Edward",male,55.0,0,0,680,50.0000,C39,S,0,0.0,London / Birmingham,0
846,174,1,"Kent, Mr. Edward Austin",male,58.0,0,0,11771,29.7000,B37,C,0,258.0,"Buffalo, NY",0
847,467,2,"Kantor, Mrs. Sinai (Miriam Sternin)",female,24.0,1,0,244367,26.0000,0,S,12,0.0,"Moscow / Bronx, NY",1
848,1112,3,"Peacock, Miss. Treasteall",female,3.0,1,1,SOTON/O.Q. 3101315,13.7750,0,S,0,0.0,0,0


In [17]:
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train[["age","fare","body"]]
test_x = test[["age","fare","body"]]
train_y = train[["survived"]]
test_y = test[["survived"]]

In [18]:
model = LogisticRegression().fit(train_x, train_y)

C:\Users\sfennelly\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [24]:
predictions = model.predict(test_x)

In [25]:
rmse, mae, r2 = eval_metrics(test_y,predictions)

In [28]:
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("r2", r2)
mlflow.log_metric("mae", mae)

In [29]:
mlflow.sklearn.log_model(model, "model")

ModelInfo(artifact_path='model', flavors={'python_function': {'model_path': 'model.pkl', 'loader_module': 'mlflow.sklearn', 'python_version': '3.8.8', 'env': 'conda.yaml'}, 'sklearn': {'pickled_model': 'model.pkl', 'sklearn_version': '1.0.2', 'serialization_format': 'cloudpickle'}}, model_uri='runs:/1332375817914a4f9227680219560742/model', model_uuid='3af0d3b840ca4a97bfd272f58727e05b', run_id='1332375817914a4f9227680219560742', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-03-10 01:51:03.457316')